### DISCUSSION POINT: ###

IF TO USE THIS HOUSING DATA
1. **Use postcode or LGA** - LGA usually covers larger area than one postcode, which might mean less pointy data, but makes better sense to readers; on the other hand one postcode corresponds to multiple suburbs, can be hard to describe (below preliminary analysis was done using postcode for simplicity). 
Postcode to LGA Mapping [here](https://www.dva.gov.au/sites/default/files/Providers/nsworp.pdf).


2. **Limit to Sydney Region or whole NSW** - the dataset contains data for all NSW regions (central coast, Wollogong etc.). Are we going to limit our analysis to Greater Syndey Region only or not? If so, need to figure out a method tease out Sydney LGAs / postcodes - scraping some gov. table (above mapping for example) and use the join method.


3. **What variables from census to inclue and how** intuitively income, employment status, age, household size etc. We can merge in the master df and use RFE to decide which one is relevant. But the problem is - census data is categorical, you'll see what I mean by look at the example below of income - each bucket is one column, how to do modellilng using these bucket-level variables?

**DATA SOURCE:**

[NSW Housing Rent and Sales](https://www.facs.nsw.gov.au/resources/statistics/rent-and-sales/back-issues)

Sales data - renamed vs. original variable names:
* <b>dwelling_type</b>: Dwelling Type
* <b>25%_price</b>: First Quartile Sales Price (AUD 000s)
* <b>50%_price</b>: Median Sales Price (AUD 000s)
* <b>75%_price</b>: Third Quartile Sales Price (AUD 000s)
* <b>mean_prce</b>: Mean Sales Price (AUD 000s)
* <b>sales_no</b>: Number of Sales
* <b>Qdelta_median</b>: Qtly change in Median
* <b>Adelta_median</b>: Annual change in Median
* <b>Qdelta_count</b>: Qtly change in Count
* <b>Adelta_count</b>: Annual change in Count

# Cleaning Sales Data

### 1-1. Data cleaning and preparation

Had to change some code to work for both 2017_2018 fiiles and 2019_2021 as there were minor differences. 

In [1]:
import glob

import pandas as pd
import numpy as np 
 import seaborn as sns
%matplotlib inline


### N.B. 
therer seems to be a probblem with reading the csv files. The first three have bad headers. I've tested this with removing the first three called files and still had the same problem. There are some files in the earlier years that have a mix of column headers 

the earlier files had no keys (issue number) so we had to change the file name manually. The ssue numbers were containied in the first page of the files. 

help from https://stackoverflow.com/questions/14008440/how-to-extract-numbers-from-filename-in-python

In [2]:
import re

In [3]:
def salesCleanFn(dataFolderString,osString):
    
    #__________________________________________________________
    # If operating system is windows
    if osString == 'windows':
        dataDir = "Files\\"
        if dataFolderString[-1] != '\\':
        #if the dataFolderString does not have a forward slash, add a forward slash to the string
            fileNames = glob.glob(dataDir+dataFolderString+'\\'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')

    #__________________________________________________________
    # If operating system is mac / linux 
    else:
        dataDir = "Files/"
        if dataFolderString[-1] != '/':
            fileNames = glob.glob(dataDir+dataFolderString+'/'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')
#     print(fileNames)
    
    #__________________________________________________________
    # Looping through the fileNames to read the excel sheets. 
    frames = []
    masterDF = []

    for i, fileString in enumerate(fileNames):
        for j in range(0,8):
            df = []
            df = pd.read_excel(fileString, sheet_name="Postcode", na_values='-', header=j)

            if df.columns[0] == 'Postcode': # ...if the 'header' parameter has the correct j value...

                # _____________________________________________________________________________________
                # adding additional columns
                regex = re.compile(r'\d+') #finds all numbers in string
                fileNumbers = regex.findall(fileString) #only works if the format of the 
                # filename is consistent. Stores the numbers in a list.
                
#                 print(fileNumbers)
                df['key'] = 's'+fileNumbers[2] # fileNumbers type = string
                year = fileNumbers[3] #type = string
                
                # the following statement searches for the month in the filename
                # if the find() function does not find the month, it returns '-1'
                # thus the use of !=-1. 
                if fileString.find('mar') !=-1 or fileString.find('Mar') !=-1:
                    quarter = 'Q1'
                elif fileString.find('jun') !=-1 or fileString.find('Jun') !=-1:
                    quarter = 'Q2'
                elif fileString.find('sep') !=-1 or fileString.find('Sep') !=-1:
                    quarter = 'Q3'
                elif fileString.find('dec') !=-1 or fileString.find('Dec') !=-1:
                    quarter = 'Q4'
                df['time_period'] = year + ' ' + quarter

                df['year'] = year

                df['quarter'] = quarter
                
                # some of the columns in the files are not the same, so we fix them here
                column = 'Quarterly change in Median Sales Price'
                newColumns = {'Quarterly change in Median Sales Price':'Qtly change in Median',
                             'Annual change in Median Sales Price':'Annual change in Median',
                             'Quarterly change in Count':'Qtly change in Count'}
        
                if column in df.columns:
                    df.rename(columns = newColumns, inplace= True)
                
                # finally, putting the DF into a list, frames
                frames.extend([df])
                
               


    # _____________________________________________________________________________________
    # putting all the DFs (frames) together to get a master DF
    masterDF = pd.concat(frames)
    # General cleaning
    rename_cols= {'Postcode':'postcode', 
             'Dwelling Type':'dwelling_type', 
             "First Quartile Sales Price\n$'000s" : '25%_price',
             "Median Sales Price\n$'000s" : 'median_price', 
             "Third Quartile Sales Price\n'000s" : '75%_price',
             "Mean Sales Price\n$'000s" : 'mean_price',
             'Sales\nNo.':'sales_no',
             'Qtly change in Median':'Qdelta_median',
             'Annual change in Median':'Adelta_median',
             'Qtly change in Count':'Qdelta_count',
             'Annual change in Count':'Adelta_count'}
    
    masterDF.rename(columns=rename_cols, inplace=True) #rename the columns for easier referencing



    masterDF = masterDF.drop(columns=['25%_price', '75%_price'], axis=1) # dropping unwanted columns
    
    masterDF.loc[masterDF['sales_no'].isnull(), 'sales_no'] = 5.0 #imputing NAN values. 5 is median of 0 and 10 being the 
    # range for null values in the dataset. 
    
    # fixing the NAN values in the median and mean columns 
    keys = list(masterDF['key'].unique())

    for k in keys:
    # Total
    # median
        k_impMedianTotal = masterDF.loc[(masterDF['median_price'].notna()) & 
                             (masterDF['dwelling_type']=='Total') &
                             (masterDF['key']==k),
                             'median_price'].median() # calculate imputer value 

        masterDF.loc[(masterDF['median_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Total') &
                     (masterDF['key']==k),
                     'median_price']=k_impMedianTotal #impute

    # mean
        k_impMeanTotal = masterDF.loc[(masterDF['mean_price'].notna()) & 
                             (masterDF['dwelling_type']=='Total') &
                             (masterDF['key']==k),
                             'median_price'].median()

        masterDF.loc[(masterDF['mean_price'].isnull()) & 
                 (masterDF['dwelling_type']=='Total') &
                 (masterDF['key']==k),
                 'mean_price']=k_impMeanTotal #impute
#         print(k_impMeanTotal)
#         print('')
#         print(k)

    # Strata
    # median
        k_impMedianStrata = masterDF.loc[(masterDF['median_price'].notna()) & 
                             (masterDF['dwelling_type']=='Strata') &
                             (masterDF['key']==k),
                             'median_price'].median()


        masterDF.loc[(masterDF['median_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Strata') &
                     (masterDF['key']==k),
                     'median_price']=k_impMedianStrata

    # mean
        k_impMeanStrata = masterDF.loc[(masterDF['mean_price'].notna()) & 
                             (masterDF['dwelling_type']=='Strata') &
                             (masterDF['key']==k),
                             'mean_price'].median()

        masterDF.loc[(masterDF['mean_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Strata') &
                     (masterDF['key']==k),
                     'mean_price']=k_impMeanStrata

    # Non-Strata
    # median
        k_impMedianNonStrata = masterDF.loc[(masterDF['median_price'].notna()) & 
                             (masterDF['dwelling_type']=='Non Strata') &
                             (masterDF['key']==k),
                             'median_price'].median()

        masterDF.loc[(masterDF['median_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Non Strata') &
                     (masterDF['key']==k),
                     'median_price']=k_impMedianNonStrata

    # mean
        k_impMeanNonStrata = masterDF.loc[(masterDF['mean_price'].notna()) & 
                             (masterDF['dwelling_type']=='Non Strata') &
                             (masterDF['key']==k),
                             'mean_price'].median()

        masterDF.loc[(masterDF['mean_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Non Strata') &
                     (masterDF['key']==k),
                     'mean_price']=k_impMeanNonStrata
        continue

    masterDF.loc[masterDF['sales_no'] == 's', 'sales_no'] = 20.0 # Replace 's' with the median of 
    # 10 and 30 since there are quite a few

    masterDF['sales_no'] = masterDF['sales_no'].astype(float) # Cast type as float

    total = masterDF.loc[masterDF['dwelling_type']=='Total'] # Separate dwelling types
    strata = masterDF.loc[masterDF['dwelling_type']=='Strata']
    nstrata = masterDF.loc[masterDF['dwelling_type']=='Non Strata'] 

    return masterDF, total, strata, nstrata

In [4]:
salesOld, salesOld_total, salesOld_strata, salesOld_nStrata  = salesCleanFn('Sales/2017_2018', 'windows')

In [5]:
salesOld.to_csv('Files/sales_2017_2018')

In [6]:
salesOld_total.to_csv('Files/salesTotal_2017_2018')

In [7]:
salesOld_strata.to_csv('Files/salesStrata_2017_2018')

In [8]:
salesOld_nStrata.to_csv('Files/salesNstrata_2017_2018')

In [9]:
salesNew, salesNew_total, salesNew_strata,  salesNew_nStrata =  salesCleanFn('Sales/2019_2021', 'windows')

In [10]:
salesNew.to_csv('Files/sales_2019_2021')

In [11]:
salesNew_total.to_csv('Files/salesTotal_2019_2021')

In [12]:
salesNew_strata.to_csv('Files/salesStrata_2019_2021')

In [13]:
salesNew_nStrata.to_csv('Files/salesNstrata_2019_2021')

In [14]:
salesNew

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,Adelta_median,Qdelta_count,Adelta_count,key,time_period,year,quarter
0,2000,Total,1160.0,1348.0,103.0,-0.0169,-0.1375,-0.1043,-0.1488,s128,2019 Q1,2019,Q1
1,2000,Non Strata,680.0,720.0,5.0,NaN,NaN,NaN,NaN,s128,2019 Q1,2019,Q1
2,2000,Strata,1135.0,1322.0,101.0,-0.0340,-0.0920,-0.0734,-0.1062,s128,2019 Q1,2019,Q1
3,2007,Total,641.0,517.0,20.0,-0.0642,-0.1097,-0.1333,-0.3158,s128,2019 Q1,2019,Q1
4,2007,Strata,641.0,517.0,20.0,-0.0642,-0.1097,-0.1333,-0.3158,s128,2019 Q1,2019,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,2877,Non Strata,195.0,212.0,20.0,-0.0714,0.3636,0.5385,4.0000,s136,2021 Q1,2021,Q1
1423,2879,Total,760.5,760.5,5.0,NaN,NaN,NaN,NaN,s136,2021 Q1,2021,Q1
1424,2879,Non Strata,843.0,888.0,5.0,NaN,NaN,NaN,NaN,s136,2021 Q1,2021,Q1
1425,2880,Total,120.0,146.0,107.0,-0.0323,-0.0400,0.1146,0.5070,s136,2021 Q1,2021,Q1


In [15]:
salesOld

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,Adelta_median,Qdelta_count,Adelta_count,key,time_period,year,quarter
0,2000,Total,1350.0,1516.328059,135.0,0.1345,0.4746,-0.3250,-0.3112,s122,2017 Q3,2017,Q3
1,2000,Strata,1350.0,1516.328059,135.0,0.1431,0.5553,-0.3182,-0.2703,s122,2017 Q3,2017,Q3
2,2007,Total,817.5,804.448333,36.0,-0.1090,-0.1572,-0.4462,-0.3455,s122,2017 Q3,2017,Q3
3,2007,Strata,817.5,804.448333,36.0,-0.0815,-0.1528,-0.4194,-0.3208,s122,2017 Q3,2017,Q3
4,2008,Total,995.0,1061.807024,41.0,-0.0005,0.0205,-0.1087,-0.1277,s122,2017 Q3,2017,Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,2877,Non Strata,194.0,197.000000,20.0,0.2360,0.2263,0.2000,-0.2941,s127,2018 Q4,2018,Q4
1368,2880,Total,133.0,138.000000,81.0,0.1667,0.4301,0.0658,-0.1735,s127,2018 Q4,2018,Q4
1369,2880,Non Strata,133.0,138.000000,81.0,0.1667,0.4301,0.0658,-0.1735,s127,2018 Q4,2018,Q4
1370,3644,Total,680.0,680.000000,5.0,NaN,NaN,NaN,NaN,s127,2018 Q4,2018,Q4


# CLEANING RENT DATA

### cleaning function

import os

os.path.join('files','sales')

will put the appropriate slashes depending on your OS 

In [19]:
def rentCleanFn(dataFolderString,osString):
        
    #__________________________________________________________
    # If operating system is windows
    if osString == 'windows':
        dataDir = "Files\\"
        if dataFolderString[-1] != '\\':
        #if the dataFolderString does not have a forward slash, add a forward slash to the string
            fileNames = glob.glob(dataDir+dataFolderString+'\\'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')

    #__________________________________________________________
    # If operating system is mac / linux 
    else:
        dataDir = "Files/"
        if dataFolderString[-1] != '/':
            fileNames = glob.glob(dataDir+dataFolderString+'/'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')
#     print(fileNames)
    
    #__________________________________________________________
    # Looping through the fileNames to read the excel sheets. 
    frames = []
    masterDF = []
    
    for i, fileString in enumerate(fileNames):
        for j in range(0,8):
            df = []
            df = pd.read_excel(fileString, sheet_name="Postcode", na_values='-', header=j)

            if df.columns[0] == 'Postcode': # ...if the 'header' parameter has the correct j value...

                # adding additional columns
                regex = re.compile(r'\d+') #finds all numbers in string
                fileNumbers = regex.findall(fileString)
                
                df['key'] = 'r'+fileNumbers[2] # fileNumbers type = string
    
                
        
                # some of the columns in the files are not the same, so we fix them here
                column = 'Bedroom Numbers'
                newColumns = {'Bedroom Numbers':'Number of Bedrooms'}
        
                if column in df.columns:
                    df.rename(columns = newColumns, inplace= True)
                
                
                
                frames.extend([df])# putting the DF into a list, frames

            
              



    masterDF = pd.concat(frames)
    
    # droppinig this column as we've confirmed there's an issue with the raw csv file. 
    if 'Unnamed: 10' in masterDF.columns:
        masterDF=  masterDF.drop(columns='Unnamed: 10')

    # Drop unwanted columns
    masterDF = masterDF.drop(columns=['First Quartile Weekly Rent for New Bonds\n$',
                          'Third Quartile Weekly Rent for New Bonds\n$'],
                axis=1)
    
    # Rename columns
    rename_cols= {'Postcode':'postcode',
                  'Dwelling Types':'dwelling_type', 
                  'Number of Bedrooms':'bed_number',
                  'Median Weekly Rent for New Bonds\n$': 'median_rent_newb',
                  'New Bonds Lodged\nNo.' : 'new_bonds_no',
                  'Total Bonds Held\nNo.': 'total_bonds_no',
                  'Quarterly change in Median Weekly Rent':'Qdelta_median_rent',
                  'Annual change in Median Weekly Rent':'Adelta_median_rent',
                  'Quarterly change in New Bonds Lodged':'Qdelta_new_bonds',
                  'Annual change in New Bonds Lodged':'Adelta_new_bonds'}
    
    masterDF.rename(columns=rename_cols,inplace=True)

    masterDF_ag = masterDF.loc[(masterDF['bed_number']=='Total') & (masterDF['dwelling_type']=='Total')]
    masterDF_ag = masterDF_ag.drop(columns=['bed_number','dwelling_type'], axis=1)
    
    # Impute 's' in 'new_bonds_no' and 'total_bonds_no' with 20
    masterDF_ag.loc[masterDF_ag['new_bonds_no']=='s','new_bonds_no'] = 20.0
    masterDF_ag.loc[masterDF_ag['total_bonds_no']=='s', 'total_bonds_no'] = 20.0

    # Impute na in 'new_bonds_no' and 'total_bonds_no' with 5
    masterDF_ag.loc[masterDF_ag['new_bonds_no'].isnull(),'new_bonds_no'] = 5.0
    masterDF_ag.loc[masterDF_ag['total_bonds_no'].isnull(), 'total_bonds_no'] = 5.0

    # Cast both variables as float (was object)
    masterDF_ag['new_bonds_no'] = masterDF_ag['new_bonds_no'].astype(float)
    masterDF_ag['total_bonds_no'] = masterDF_ag['total_bonds_no'].astype(float)

    # Impute na in 'median_rent' with median of the column
    masterDF_ag['median_rent_newb'].fillna(masterDF_ag['median_rent_newb'].median(), inplace=True)
    

    # Set postcode as index
    
    masterDF_ag = masterDF_ag.set_index('postcode')
    return masterDF_ag

In [20]:
rentNew = rentCleanFn('Rent/2019_2021', 'windows')


time taken to run the function: 158.8437283039093


In [21]:
rentOld  = rentCleanFn('Rent/2017_2018', 'windows')

102.35191702842712


In [22]:
rentOld.to_csv('Files/rent_2017_2018')

In [23]:
rentNew.to_csv('Files/rent_2019_2021')

### (raw) hard code method

**NOTE:**

Note that an alarming 3/4 of the data has null values. This is because the data is broken down to very granular level - first by dwelling type (Total, house, townhouse, flat/unit, other) and then by bed_numbers (see below cell).m


By aggregating the data, we're able to bring down the proportion of na from 3/4 to around 1/3. But there's still need for imputation. According to the data interpretation note:

<em><b>"For confidentiality, we don't report rents in any geographical area where the number of new bonds is 10 or less (shown as na). Statistics calculated from sample sizes between 10 an 30 are shown by an 's' in the relevant table"</b></em>

<b>IMPUTATION</b>
* For 'new_bonds_no' and 'total_bonds_no' columns:
    * Impute na with 5
    * Impute s with 20
    
* For 'median_rent_newb' column
    * Impute na with median of rents of all POAs

### (raw) sales data original cleaning process, hard code method

Note that each postcode has 3 rows - Total, Strata, and Non-Strata. We'll later separate them into three dataframes.

Sales number was read into the dataframe as string because accordingly to the Explanatory note "statistics calculated from sample sizes between 10 and 30 are shown by an ‘s’ in the relevant table.  We suggest these data are treated with caution, particularly when assessing quarterly and annual changes."

### (raw) 1-2 Exploratory and descriptive analysis

#### A. <u>Total level trends - Number of houses sold</u> ####

Key observations:
* Total sales started decline in Q1 2020 when COVID first struck, reaching a low point in Jun-20
* However, bounce back was quick to come in Q3'20 and achieved a 200% growth vs. SQLY in Q4'20

To investigate more:
* Difference between Strata and Non-Strata houses - *Strata property are mostly apartment and townhouse; while non-Strata are more likely to be houses - will explain more in the price section*
* Calculate quarter-on-quarter growth rate -> show off of skills hehee

**CR**: for the first dot point, the trend should be starting at Q3, not Q1. There is a decline in Q1 and Q2. 


#### B. <u>Total level trends - average house price</u> ####
Key observations:
* Total (Strata + non-Strata) average price has been **trending up since Q1 2020** despite COVID
* And this upward trend has been **driven by non-Strata houses**, the price of which have rocketed since Q2 2020
* Strata properties (more likely to be apartment units/condos, terrace houses with shared common areas) on the contrary saw moderate increase in price in Q2'20 to Q4 then falling flat

See [here](https://www.macquarie.com.au/home-loans/strata-properties-pros-and-cons.html) for more information on difference of Strata and non-strata property. 

**THIS IS IMPORTANT: suggesting that Strata and non-Strata house prices behave very diffirently and should probably be looked at separately in later regression analysis**

**CR** we want to find a way to get a map of sydney, link the postcodes, that way we can have a heatmap for better visualisation of sales around Sydney. Perhaps finding a website that gets the boundaries of the city with the post code. Is there a way to extract those boundaries? 

will have a look into this link : https://github.com/chrisberkhout/jvectormap_data_au

#### C. <u>Suburb level (Q1 2021) - hottest and most expensive suburbs</u> ####

**TO BE DONE:**
* Scrape a table somewhere online to map postcode to the name of suburbs
* Or make a decision to use the LGA tab of the raw data sheets (instead of the postcode sheet used here)
* Investigate map visualisation

Roughly:
* **2250**: part of Gosford LGA (Central Coast Region)
* **2155**: part of Cherrybrook LGA 
* **2540**: Culburra LGA (Illawara Region)
* **2170**: part of Liverpool LGA
* **2650**: part of Junee LGA (Murrumbidgee Region)
* **2251**: part of Wyong LGA (Central Coast Region)
* **2560**: part of Campbelltown LGA 
* **2259**: part of Wyong LGA (Central Coast Region)
* **2145**: part of Holroyd LGA 
* **2444**: part of Port Macquarie LGA (Mid North Coast Region)

Look at how the price in these areas has changed.

Next, look at average price by postcode.

They are:
* **2220:** Hurstville & Hurstville Grove
* **2027:** Waverly LGA - Darling Point, Edgecliff & Point Piper (in
* **2092:** Manly Warringah LGA - Seafort
* **2030:** Waverly LGA - Rose Bay North, Vaucluse, Watsons Bay
* **2107:** Manly Warringah LGA - NEWPORT BEACH, AVALON, AVALON BEACH, BILGOLA, CLAREVILLE, WHALE BEACH

### (raw)1-3 Join Census Data

**KS**: perhaps change the income brackets to low medium and high instead? 

**CR**: should consider ordering the column incomes from lowest to highest or the other way around.

**CR**: joining two DFs together, matching them by postcode. 

### (raw) 2. Rent data

<u>USAGE POTENTIALS:</u>
1. Analyse correlation of rental price and sales price 

### bonds yield